# Training of sentiment analysis model with the embeddings computed on previous notebook

41k spanish tweets about football, 15k spanish amazon reviews, ~4k spansih movie reviews, 5k english movie reviews and ~7k spanish UOC's tweets about politics.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 250)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import pickle

In [2]:
embeddings_path='../../data/sentiment_datasets_embeddings/'

#EN_MOVIES
with open(embeddings_path+'IMDB_Dataset_embeddings.npy', 'rb') as f:
     IMDB_Dataset_embeddings_tmp=np.load(f)
with open(embeddings_path+'IMDB_Dataset_labels.npy', 'rb') as f:
     IMDB_Dataset_labels_tmp=np.load(f)
        
#ES_MOVIES  
with open(embeddings_path+'es_movies_embeddings.npy', 'rb') as f:
     es_movies_embeddings=np.load(f)
with open(embeddings_path+'es_movies_labels.npy', 'rb') as f:
     es_movies_labels=np.load(f)
        
#ES_REVIEWS
with open(embeddings_path+'es_reviews_embeddings.npy', 'rb') as f:
     es_reviews_embeddings=np.load(f)
with open(embeddings_path+'es_reviews_labels.npy', 'rb') as f:
     es_reviews_labels=np.load(f)
        
#ES_TWEETS
with open(embeddings_path+'spanish_tweets_sentiment_embeddings.npy', 'rb') as f:
     spanish_tweets_sentiment_embeddings=np.load(f)
with open(embeddings_path+'spanish_tweets_sentiment_labels.npy', 'rb') as f:
     spanish_tweets_sentiment_labels=np.load(f)
        
#UOC_TWEETS
with open(embeddings_path+'UOC_tweets_embeddings.npy', 'rb') as f:
     UOC_tweets__embeddings=np.load(f)
with open(embeddings_path+'UOC_tweets_labels.npy', 'rb') as f:
     UOC_tweets_labels=np.load(f)

In [7]:
es_movies_embeddings.shape

(3872, 768)

In [36]:
#En el dataset de reviews en inglés hay más de las que quiero así que elijo 5k al azar (balanceado)
np.random.seed(6)
indices=np.random.randint(0,len(IMDB_Dataset_labels_tmp),5000)
pd.Series(IMDB_Dataset_labels_tmp[indices]).value_counts() #2.5k negativas  y 2.5k positivas
IMDB_Dataset_embeddings=IMDB_Dataset_embeddings_tmp[indices,:]
IMDB_Dataset_labels=IMDB_Dataset_labels_tmp[indices]

In [37]:
#Junto todo
embeddings=np.concatenate((IMDB_Dataset_embeddings,es_movies_embeddings,es_reviews_embeddings,
                           spanish_tweets_sentiment_embeddings,UOC_tweets__embeddings))
labels=np.concatenate((IMDB_Dataset_labels,es_movies_labels,es_reviews_labels,
                       spanish_tweets_sentiment_labels,UOC_tweets_labels))
embeddings.shape,labels.shape

((73191, 768), (73191,))

In [38]:
#Separo datos en 80% entreno 20% test
x_train,x_test,y_train,y_test = train_test_split(embeddings,labels,stratify=labels,test_size=0.2,random_state=42)

In [39]:
pd.Series(y_train).value_counts() #Esta ligeramente desbalanceado, no mucho, así que sigo con esto

 0    20460
 1    19880
-1    18212
dtype: int64

In [41]:
%%time
#Cuanto mayor es C, más tarda en entrenar
for c in [0.1,1,10]:    
    sentiment_model = LogisticRegression(C=c,max_iter=1500,n_jobs=-1)
    sentiment_model.fit(x_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, sentiment_model.predict(x_test)))) 
#Me quedo con C=1 ya que da buenos resultados y con C=10 puede haber más overfitting.
#Aunque y_test nunca lo ha visto el modelo, sí que son datos provenientes del mismo dataset y aunque haya intentando variar,
#no son represetnativos de todos los tweets que usaré. Además, la mejora de precisión es despreciable

Accuracy for C=0.1: 0.7555160871644238
Accuracy for C=1: 0.7615957374137577
Accuracy for C=10: 0.7624154655372635
Wall time: 7min 50s


In [42]:
#Pruebo random forest a ver si hay mejora sustancial
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(x_train, y_train)
print(accuracy_score(y_test, rf.predict(x_test)))
#No la mejora no es significativa así que no me preocupo en buscar hyperparámetros. Me quedo con regriós logística que es simple
#y efectiva

0.7542181843022064


In [43]:
%%time
#Una vez decidido el hiperparámetro, entreno con todos los datos
sentiment_model = LogisticRegression(C=1,max_iter=1500,n_jobs=-1)
sentiment_model.fit(embeddings, labels)

Wall time: 3min 25s


LogisticRegression(C=1, max_iter=1500, n_jobs=-1)

In [44]:
#Esta precisión no es fiable porque son datos utilizados para el entreno pero vemos que la precisión no ha mejorado mucho
#respecto a antes así que no parece haber overfitting
print(accuracy_score(y_test, sentiment_model.predict(x_test)))
print(accuracy_score(IMDB_Dataset_labels_tmp[-indices], sentiment_model.predict(IMDB_Dataset_embeddings_tmp[-indices,:])))
#Todo el resto de pelis no vistas las clasifica bien

0.7717740282806203
0.8012


In [40]:
#Algunas pruebas rápidas
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
prueba=np.array(model.encode(list(['Estoy hasta las narices, no puedo más',
                                   'Me ha salido un día bastante bueno, a seguir trabajando',
                                   'El otro día me encontré un erizo',
                                   'El otro día me encontré un erizo y me mordió',
                                  'El otro día me encontré un erizo y me mordió. Sin embargo, no me quejo, estoy bien con mi vida',
                                   'Im quite a bit nervous',
                                   'Best day of my life',
                                   'How dare you to do this to me?',
                                  'El otro día fui al cine después de tanto tiempo... La peli estuvo bien, sin más.']), 
                                            show_progress_bar=True,batch_size=128))
sentiment_model.predict(prueba)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([-1,  1,  0, -1,  0,  0,  1, -1,  0], dtype=int64)

In [45]:
#Guardo el modelo
pickle.dump(sentiment_model, open('sentiment_model.sav', 'wb'))